#  Transforming Collections to Linked Art - Indianapolis Museum of Art

## Introduction

[Linked Art](https://linked.art) is a community working together to create a shared Model based on Linked Open Data to describe Art. A number of exemplars will be published to demonstrate the processes involved in producing Linked Art JSON-LD, and also the potential applications of Linked Art, on the theme of:
- `Transformation` - Documented transformation process - using code, documentation and possibly visualisation
- `Reconciliation` - Documented reconciliation process - matching data with an external identifier source
- `Visualisation` - Documented transformation of Linked Art JSON-LD to data visualisation

This exemplar is concerned with `Transformation` - the transformation process, from collections data to Linked Art JSON-LD.

## Aim of the Notebook
The aim of the notebook is to demonstrate how easy it is to transform collections data to Linked Art JSON-LD.
## How
The notebook provides a documented, interactive code example of the transformation process, from collections data to Linked Art using data from the [Indianapolis Museum of Art (IMA)]((https://discovernewfields.org/)). 

## Input Data
The [Indianapolis Museum of Art (IMA)]((https://discovernewfields.org/)) has transformed a sample of its collections data to Linked Art JSON-LD:
- available at https://github.com/IMAmuseum/LinkedArt 
- sourced from EMu in XML format (EMu source: Catalogue, Rights, Narratives, and Locations modules)
- [XML files](https://github.com/IMAmuseum/LinkedArt/blob/master/XML)
 - [ObjectsSample XML file](https://github.com/IMAmuseum/LinkedArt/blob/master/XML/ObjectsSample.xml) [raw file](https://raw.githubusercontent.com/IMAmuseum/LinkedArt/master/XML/ObjectsSample.xml)

<div class='alert alert-block alert-info'>An XML file is used as data input for the notebook, and as a miminum the ObjectsSample XML file should be downloaded.</div>

## Attribution

- The notebook's coded transformations are based on the IMA's [XSLT file](https://github.com/IMAmuseum/LinkedArt).
- The Linked Art data model documentation has been sourced from the [Linked Art website](https://linked.art)
- The IMA data has been sourced from the [IMA GitHub repository](https://github.com/IMAmuseum/LinkedArt)

## Transformation Steps

### 1. Import What We Need for Notebook and Define Variable
- Import Python libraries

In [74]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

from ipywidgets import Layout
from ipywidgets import FileUpload

try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import IFrame

   
try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json 
    
    

try:
    import requests
except:
    !pip install requests
    import requests

        
def widgeText(desc, jdoc, ht):
    widg = widgets.Textarea(
        value=json.dumps(jdoc, indent=2),
        placeholder="",
        description=desc,
        disabled=False,
        layout=Layout(width='100%', height=ht))
    return widg

def write2file(filename, obj):
    # write to file
    f = open("./data/ima/output/json/" + filename + ".json", "w")
    f.write(json.dumps(obj, indent=2))
    f.close() 

#  baseURI for JSON-LD document e.g. https://data.discovernewfields.org/
baseURI = "https://data.discovernewfields.org/"

### 2. Upload XML File
- Choose a file on your local system to upload and transform to Linked Art
- The IMA files are available to download from: https://github.com/IMAmuseum/LinkedArt/tree/master/XML
- Select a file by clicking on the "Select XML file" button

In [3]:
upload = FileUpload(accept='.xml', multiple=False, description='Select XML file')

In [5]:
display(HTML("<div class='alert alert-block alert-info'>Please select a file to transform</div>"))
display(upload)

FileUpload(value={'ObjectsSample.xml': {'metadata': {'name': 'ObjectsSample.xml', 'type': 'text/xml', 'size': …

In [6]:
obj = False
# get content from uploaded file 
for uploaded_filename in upload.value:
    content = upload.value[uploaded_filename]['content']
    obj = xmltodict.parse(content) 
    
if obj == False:
    display(HTML("<div class='alert alert-block alert-danger'>Please select a file to transform</div>"))
else:
    display(HTML("<div class='alert alert-block alert-success'>File uploaded</div>"))

### 3. Select Catalogue Item to Transform
The next step will transform the contents of the uploaded XML file into a dropdown list of artworks.

In [7]:
allObjects = obj["table"]["tuple"]

#object dropdown options
objOptions = []
objOptions.append(("Please select an object",''))

for obj in allObjects:
    title = irn = ""
    # define properties variables  
    for prop in obj["atom"]:      
        propName = prop["@name"]     
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]  
            if propName == "TitMainTitle":
                title = prop["#text"]
            
    objOptions.append((irn + ' : ' + title,irn))
    
selectObject = widgets.Dropdown(options=objOptions, description='Select Object')

In [9]:
display(HTML("<div class='alert alert-block alert-info'>Select an artwork from the dropdown list to transform the catalogue data to Linked Art JSON-LD</div>"))

display(selectObject)

Dropdown(description='Select Object', index=45, options=(('Please select an object', ''), ('1032 : long-neck v…

In [10]:
selectedIRN = selectObject.value

if selectedIRN == "":
    display(HTML("<div class='alert alert-block alert-danger'>Please select an artwork to transform</div>"))
else:
    display(HTML("<div class='alert alert-block alert-success'>Artwork selected:" + selectObject.options[selectObject.index][0] + "</div"))  


### Artwork as Python dictionary
The following Python dictionary is for the selected artwork, and will be used to transform the collection data for the artwork to JSON-LD:

In [11]:
selectedObject = {}
for obj in allObjects:
    for prop in obj["atom"]:         
        if prop["@name"] == "irn":
            irn = prop["#text"]
            if irn == selectedIRN:
                selectedObject = obj
               
display(selectedObject)

OrderedDict([('atom',
              [OrderedDict([('@name', 'irn'),
                            ('@type', 'text'),
                            ('@size', 'short'),
                            ('#text', '82696')]),
               OrderedDict([('@name', 'AdmPublishWebNoPassword'),
                            ('@type', 'text'),
                            ('@size', 'short'),
                            ('#text', 'Yes')]),
               OrderedDict([('@name', 'TitAccessionNo'),
                            ('@type', 'text'),
                            ('@size', 'short'),
                            ('#text', '2013.443A-E.4')]),
               OrderedDict([('@name', 'TitPreviousAccessionNo'),
                            ('@type', 'text'),
                            ('@size', 'short'),
                            ('#text', 'TR11707A-E.4')]),
               OrderedDict([('@name', 'TitObjectStatus'),
                            ('@type', 'text'),
                            ('@size', 'short')

## 4. Build the Linked Art JSON-LD file <a id="build"/>

The following steps will transform the catalogue data for the selected artwork to Linked Art JSON-LD. The transformation will be divided into sections, using different parts of the Linked Art data model.

- [Core Properties](#core)
- [Identifiers](#id)
- [Names](#names)
- [Classification](#class)
- [Home Page](#home)
- [Current Location](#loc)
- [Linguistic Objects](#ling)
- [Production](#prod)
- [Acquisition](#owner)
- [Custody](#curate)
- [Membership](#member)

In [12]:
obj = selectedObject

earliestdate = ""
latestdate = ""
datecreated = ""
    
for prop in obj["atom"]:      
        propName = prop["@name"]        
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]    
            if propName == "TitAccessionNo":
                titAccessionNo = prop["#text"]
            if propName == "TitMainTitle":
                title = prop["#text"]
            if propName == "TitObjectType":
                titleObjectType = prop["#text"]
            if propName == "SumCreditLine":
                SumCreditLine = prop["#text"]
            if propName == 'CreProvenance':
                provenance = prop["#text"]
            # dates
            if propName == "CreDateCreated":
                datecreated = prop["#text"]
            if propName == "CreEarliestDate":
                earliestdate = prop["#text"]
            if propName == "CreLatestDate":
                latestdate = prop["#text"]
            if propName == "TitAccessionDate":
                TitAccessionDate = prop["#text"]
            if propName == 'TitObjectStatus':
                TitObjectStatus = prop["#text"]
            if propName == "PhyCollectionArea":
                PhyCollectionArea = prop["#text"]
                
                
                

### 4.1 Core Properties <a id='#core'>
<a id='core_properties'></a>
[Linked Art Data Model documentation](https://linked.art/model/base/#core-properties)

There are a few core properties that every resource should have for it to be a useful part of the world of Linked Open Data:

- `@context`
- `id`
- `type`
- `_label`
    
---

#### IMA Data Mapping

The `id` is a URL and has been created from the `irn` value together with a URL prefix: https://data.discovernewfields.org/

The `_label` is a human readable label, intended for developers and other people reading the data. The value is taken from the `TitMainTitle` property.
    
--- 
    
[Back to build menu](#build)

In [13]:
def objCore(obj, baseURI, irn, title):
    core = {}

        # minimum Linked Art properties
    core["@context"] = "https://linked.art/ns/v1/linked-art.json"
    core["id"] = baseURI + "object/" + irn
    core["type"] = "HumanMadeObject"
    if "title" in vars():
        core["_label"] = title 
    
    return core

In [14]:
core = objCore(obj,baseURI,irn,title)
min = {}
min.update(core)
write2file("min",min)
print(json.dumps(min,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes"
}


### 4.2 Identifiers <a id="id"/>

[Linked Art Data Model documentation](https://linked.art/model/base/#identifiers)

Many resources of interest are also given external identifiers, such as accession numbers for objects, ORCIDs for people or groups, lot numbers for auctions, and so forth. Identifiers are represented in a very similar way to names, but instead use the Identifier class. Identifiers will normally have a classification determining which sort of identifier it is, to distinguish between internal repository system assigned numbers from museum assigned accession numbers, for example.

As Identifiers and Names use the same `identified_by` property, the JSON will frequently have mixed classes in the array. Unlike `Names`, `Identifiers` are not part of human language and thus cannot have translations or a language associated with them.

---

[Back to build menu](#build)

In [15]:
def objId(obj, baseURI, irn, titAccessionNo):
    artwork = {}       
    artwork["identified_by"] = []          
    artwork["identified_by"].append({
        "id": baseURI + "object/" + irn + "/irn",
        "type": "Identifier",
        "_label": "IMA at Newfields Collections Database Number for the Object",
        "content": irn,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300404621",
            "type": "Type",
            "_label": "repository numbers"
                        }]
                })                 
    artwork["identified_by"].append({
        "id": baseURI + "object/" + irn + "/object-number",
        "type": "Identifier",
        "_label": "IMA at Newfields Object Number for the Object",
        "content": titAccessionNo,
        "classified_as": [{
            "id": "http://vocab.getty.edu/aat/300312355",
            "type": "Type",
            "_label": "accession numbers"
                        }]
                })
        
    return artwork

In [16]:
identifiers = objId(obj, baseURI, irn, titAccessionNo)
frag = {}
frag.update(min)
frag.update(identifiers)

write2file("ids",frag)

print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/object/82696/irn",
      "type": "Identifier",
      "_label": "IMA at Newfields Collections Database Number for the Object",
      "content": "82696",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404621",
          "type": "Type",
          "_label": "repository numbers"
        }
      ]
    },
    {
      "id": "https://data.discovernewfields.org/object/82696/object-number",
      "type": "Identifier",
      "_label": "IMA at Newfields Object Number for the Object",
      "content": "2013.443A-E.4",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
          "_label": "accession numbers"
        }
      ]
    }
  ]
}


### 4.3 Names <a id="name"/>

https://linked.art/model/base/#names

As the `_label` property is intended as internal documentation for the data, it is strongly recommended that every resource that should be rendered to an end user also have at least one specific name. The name could be for an object, a person, a group, an event or anything else. This pattern uses the `identified_by` property, with a `Name` resource. The value of the name is given in the content property of the `Name`.

It is somewhat unintuitive to think of a name as identifying the resource it is associated with, as names are typically not unique. However, as the name itself is uniquely identified rather than just an anonymous string, they are no longer a shared label and instead the particular instance of a name is uniquely associated with the resource. With this formulation, the name instance does uniquely identify the resource.

If there is more than one name given, then there should be one that is `classified_as` the primary name for use. This is done by adding the `Primary Name` (aat:300404670) term to it. There should be exactly one primary title given per language.

Names are also part of human communication, and can have the Linguistic features of the model associated with them, such as having a particular language, or having translations.

---

[Back to build menu](#build)

In [17]:
def objNames(obj, baseURI, irn, title):

    artwork = {}
    artwork["identified_by"] = []
    
    artwork["identified_by"].append({
        "id": baseURI + "object/" + irn + "/title",
        "type": "Name",
        "_label": "Primary Title for the Object",
        "content": title ,
        "classified_as": [{
        "id": "http://vocab.getty.edu/aat/300404670",
        "type": "Type",
        "_label": "preferred terms"
                        }]
                })
    try:   
        if obj["table"]["@name"] == "AltTitles":
            x = 0
            for tuple in obj["table"]["tuple"]:
                x +=1
                for atom in tuple:
                    content = ""
                    if atom["@name"] == "TitAlternateTitles":
                        content = atom["#text"]
                        artwork["identified_by"].append({
                            "id": baseURI + "object/" + irn + "/alt-title-" + x,
                            "type": "Name",
                            "_label": "Alternate Title for the Object",
                            "content": content,
                            "classified_as": [{
                                "id": "http://vocab.getty.edu/aat/300417227",
                                "type": "Type",
                                "_label": "alternate titles"}]   
                        })
    except:
        pass

    return artwork

In [18]:
names = objNames(obj, baseURI, irn, title)
frag = {}
frag.update(min)
frag.update(names)

write2file("name",frag)

print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/object/82696/title",
      "type": "Name",
      "_label": "Primary Title for the Object",
      "content": "Five Brushstrokes",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "preferred terms"
        }
      ]
    }
  ]
}


### 4.4 Classification <a id="class"/>

https://linked.art/model/base/#types-and-classifications

CIDOC-CRM is a framework that must be extended via additional vocabularies and ontologies to be useful. The provided mechanism for doing this is the classified_as property, which refers to a term from a controlled vocabulary. This is in contrast to the `type` property, which is used for CIDOC-CRM defined classes, and a few extensions as needed. 

The `classified_as` property is thus a way to be more specific about the sort of entity, while maintaining the core information as the class using type. Controlled vocabulary entries should not be used with `type`, nor classes used with `classified_as`.

While any external vocabulary of terms can be used, the Getty's Art and Architecture Thesaurus is used whenever possible for consistency and that it is already widespread in the museum domain. The set of terms that have been identified as useful are listed in the community best-practices for recommendations, and within the documentation of the model when a particular choice is essential for interoperability.

---
[Back to build menu](#build)

In [19]:
def objClass(obj, baseURI, irn, titleObjectType):  
    artwork = {}
    artwork["classified_as"] = []
    objtype = {}
    objtype["Drawings"]     = (300033973, "drawings (visual works)")
    objtype["Multimedia"]   = (300047910,"multimedia works")
    objtype["Needlework"]   = (300264072,"needlework (visual works)")
    objtype["Paintings"]    = (300033618,"paintings (visual works)")
    objtype["Pastel"]       = (300076922,"pastels (visual works)")
    objtype["Performance"]  = (300121445,"performance art")
    objtype["Photograph"]   = (300046300,"photographs")
    objtype["Prints"]       = (300041273,"prints (visual works)")
    objtype["Sculpture"]    = (300047090,"sculpture (visual works)")
    
    phyMediaCategory = ""          
    for table in obj["table"]:
        tableName = table["@name"]
        if tableName == "ObjectTypes":
            try:
                for atom in table["tuple"]:
                    if atom["atom"]["@name"] == "PhyMediaCategory":
                        phyMediaCategory = atom["atom"]["#text"]
            except:
                pass
    
    if titleObjectType.startswith("Visual Work"):
        artwork["classified_as"].append({
                    "id": "http://vocab.getty.edu/aat/300133025",
                    "type": "Type",
                    "_label": "works of art" 
                })
    
    for type in objtype:
        if type in titleObjectType:
            artwork["classified_as"].append ({
                    "id": "http://vocab.getty.edu/aat/" + str(objtype[type][0]),
                    "type": "Type",
                    "_label": objtype[type][1] 
                })
            
    if phyMediaCategory != "":  
        artwork["classified_as"].append(
                    {
                    "id": baseURI + "thesauri/type/",
                    "type": "Type",
                    "_label": phyMediaCategory
                    }
                )
       
    return artwork

In [20]:
if "titleObjectType" in vars():
    clasf = objClass(obj, baseURI, irn, titleObjectType)
frag = {}
frag.update(min)
frag.update(clasf)
write2file("class",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300133025",
      "type": "Type",
      "_label": "works of art"
    },
    {
      "id": "http://vocab.getty.edu/aat/300047090",
      "type": "Type",
      "_label": "sculpture (visual works)"
    }
  ]
}


### 4.5 Home Page <a id="home"/>

https://linked.art/model/digital/#home-page

A very common scenario is that there is a web page about the object, perhaps managed by a collections management system. For humans, this page is much more useful than the data intended for machines. It can be referenced with the `subject_of` property, and points to a `DigitalObject` which is `classified_as` a web page, or `aat:300264578`. As with digital images, the home page can have a format of "text/html" and other properties.

---
[Back to build menu](#build)

In [21]:
def objHome(obj, baseURI,irn):
    for table in obj["table"]:   
        tableName = table["@name"]
        if tableName == "Homepage":
            try:
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "EleIdentifier":
                        homepageId = atom["#text"]
            except:
                pass
        
    artwork = {}
    
    if "homepageId" in vars():
        artwork["subject_of"]=[{
        "id": "http://collection.imamuseum.org/artwork/" + homepageId,
        "type": "LinguisticObject",
        "_label": "Homepage for the Object",
        "classified_as": [
            {
            "id": "http://vocab/getty.edu/aat/300264578",
            "type": "Type",
            "_label": "Web pages (documents)"
            },
            {
            "id": "http://vocab.getty.edu/aat/300266277",
            "type": "Type",
            "_label": "home pages"
            }
        ],
        "format": "text/html"
                }]
    
    
    return artwork

In [22]:
homepage = objHome(obj,baseURI,irn)

homepageId = homepage["subject_of"][0]["id"]
frag = {}
frag.update(min)
frag.update(homepage)

write2file("homepage",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "subject_of": [
    {
      "id": "http://collection.imamuseum.org/artwork/76663",
      "type": "LinguisticObject",
      "_label": "Homepage for the Object",
      "classified_as": [
        {
          "id": "http://vocab/getty.edu/aat/300264578",
          "type": "Type",
          "_label": "Web pages (documents)"
        },
        {
          "id": "http://vocab.getty.edu/aat/300266277",
          "type": "Type",
          "_label": "home pages"
        }
      ],
      "format": "text/html"
    }
  ]
}


In [23]:
# Try to display homepage 
print(homepageId)
IFrame(homepageId, width='100%', height=500)

http://collection.imamuseum.org/artwork/76663


### 4.6 Current Location <a id="loc"/>

[Linked Art Data Model Documentation](https://linked.art/model/object/ownership/#location)

- The current location of the object is given using the `current_location` property. 
- This can give a reference to a gallery or specific part of a facility, or be used for the general address of the organization where the object is currently held. 
- There are further modeling details available about [Places](https://linked.art/model/place/) on the Linked Art website.

--- 

[Back to build menu](#build)


In [68]:
def objLocation(obj, baseURI):
    artwork = {}
    artwork["current_location"] = []
    
    level2val = level3val = ""

    for tuple in obj["tuple"]:  
        if tuple['@name'] == 'LocCurrentLocationRef':
            for atom in tuple["atom"]:
                if "#text" in atom:
                    if atom["@name"] == 'LocLevel2':
                        level2val = atom["#text"]
                        see_related_parts = False
                    
                        if atom["#text"] == 'see related parts':
                            see_related_parts = True

                    if atom["@name"] == "LocLevel3":
                        if "#text" in atom:
                            level3val = atom["#text"]
                   
# loop through again

    for tuple in obj["tuple"]:  
        if tuple['@name'] == 'LocCurrentLocationRef':
            for atom in tuple["atom"]:
                if "#text" in atom:
                    if atom["@name"] == 'LocLevel1':    
                        if atom["#text"] == 'On Loan':
                            artwork["current_location"].append(
                            {
                            "id": baseURI + "thesauri/location/on-loan",
                            "type": "Place",
                            "_label": "On Loan"
                            })
                    
                        if 'Galler' in atom["#text"] or 'Suite' in atom["#text"]:
                            artwork["current_location"].append(
                            {
                            "id": baseURI + "thesauri/location/" + level3val,
                            "type": "Place",
                            "_label": level2val,
                            "classified_as": [{
                                "id": "http://vocab.getty.edu/aat/300240057",
                                "type": "Type",
                                "_label": "galleries (display spaces)"
                                            }] 
                            })
            
                    if atom["@name"] == 'LocLevel2': 
                        if atom["#text"] == 'Efroymson Family Entrance':
                             artwork["current_location"].append(
                             { 
                             "id": baseURI + "thesauri/location/F02",
                            "type": "Place",
                            "_label": "Efroymson Family Entrance Pavilion"
                             })
                        elif atom["#text"] == 'Nature Park':
                             artwork["current_location"].append(
                             { 
                             "id": baseURI + "thesauri/location/ANP",
                            "type": "Place",
                            "_label": "Virginia B. Fairbanks Art &amp; Nature Park"
                             })
                        
                        elif atom["#text"] == 'Grounds':
                            artwork["current_location"].append(
                            { 
                             "id": baseURI + "thesauri/location/G",
                            "type": "Place",
                            "_label": "Newfields Grounds"})
                    
                        elif atom["#text"] == 'Asian Visible Storage':
                             artwork["current_location"].append(
                             { 
                             "id": baseURI + "thesauri/location/K241",
                            "type": "Place",
                            "_label": "Leah and Charles Reddish Gallery - Asian Visible Storage",
                             "classified_as": [
                                {
                                "id": "http://vocab.getty.edu/aat/300240057",
                                "type": "Type",
                                "_label": "galleries (display spaces)"
                                }]
                             })
                        elif atom["#text"] == 'Westerley':
                            artwork["current_location"].append(
                            { 
                             "id": baseURI + "thesauri/location/westerley",
                            "type": "Place",
                            "_label": "Westerley"})    
                
                        else:
                            artwork["current_location"].append({ 
                         "id": baseURI + "thesauri/location/storage",
                        "type": "Place",
                        "_label": "IMA Storage"})    
    return artwork

In [69]:
location = objLocation(obj,baseURI)

frag = {}
frag.update(min)
frag.update(location)


write2file("location",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "current_location": [
    {
      "id": "https://data.discovernewfields.org/thesauri/location/on-loan",
      "type": "Place",
      "_label": "On Loan"
    }
  ]
}


### 4.7 Statements about a Resource - Linguistic Objects <a id="ling"/>

[Linked Art Data Model Documentation](https://linked.art/model/base/#statements-about-a-resource)
    
In many cases, current data does not support the level of specificity that the full ontology allows, or the information is simply best expressed in human-readable form. For example, instead of a completely modeled set of parts with materials, many museum collection management systems allow only a single human-readable string for the "medium" or "materials statement". The same is true in many other situations, including rights or allowable usage statements, dimensions, edition statements and so forth. Any time that there is a description of the resource, with or without qualification as to the type of description, then this pattern can be used to record the descriptive text.

The pattern makes use of the `LinguisticObject` class that is used to identify a particular piece of textual content. These Linguistic Objects are then refered to by any other resource. They maintain the statement's text in the content property, and the language of the statement (if known) in the language property.

Use cases for this pattern include:

- General description of the resource
- Materials statement for an object
- Attribution statement for an image
- Biography for a person
- Dimensions statement for a part of an object    

---

[Back to build menu](#build)

In [26]:
def objLing(obj, baseURI,irn, SumCreditLine, provenance):
    artwork = {}
    artwork["referred_to_by"] = []
    if SumCreditLine != "":
        artwork["referred_to_by"].append(
                {
                "id": baseURI + "object/" + irn + "/credit-line",
                "type": "LinguisticObject",
                "_label": "IMA at Newfields Credit Line for the Object",
                "content": SumCreditLine,
                "classified_as": [
                        {
                        "id": "http://vocab.getty.edu/aat/300026687",
                        "type": "Type",
                        "_label": "acknowledgments"
                        },
                        {
                        "id": "http://vocab.getty.edu/aat/300418049",
                        "type": "Type",
                        "_label": "brief texts"
                        }]
                })
              
    if provenance != "":
        artwork["referred_to_by"].append({
                "id": baseURI + "object/" + irn + "/provenance-statement",
                    "type": "LinguisticObject",
                    "_label": "IMA Provenance Statement about the Object",
                    "content": provenance,
                    "classified_as": [
                        {
                            "id": "http://vocab.getty.edu/aat/300055863",
                            "type": "Type",
                            "_label": "provenance (history of ownership)"
                        },
                        {
                            "id": "http://vocab.getty.edu/aat/300418049",
                            "type": "Type",
                            "_label": "brief texts"
                        }
                    ]
            })
        
            
    return artwork
          

In [27]:
ling = objLing(obj,baseURI,irn, SumCreditLine, provenance)
frag = {}
frag.update(min)
frag.update(ling)


write2file("linguistic",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "referred_to_by": [
    {
      "id": "https://data.discovernewfields.org/object/82696/credit-line",
      "type": "LinguisticObject",
      "_label": "IMA at Newfields Credit Line for the Object",
      "content": "Robert L. and Marjorie J. Mann Fund, Partial Gift of the Roy Lichtenstein Foundation",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300026687",
          "type": "Type",
          "_label": "acknowledgments"
        },
        {
          "id": "http://vocab.getty.edu/aat/300418049",
          "type": "Type",
          "_label": "brief texts"
        }
      ]
    },
    {
      "id": "https://data.discovernewfields.org/object/82696/provenance-statement",
      "type": "LinguisticObject",
      "_label": "IMA Provenance Statement about the Object",
      "conte

## 4.8 Production <a id="prod"/>

[Linked Art Data Model Documentation](https://linked.art/model/object/production/)

The first activity in an object's lifecycle is its creation, or `Production`. The relationship to the object that was produced by the activity (`produced`) is added to the general activity model, along with the time, location and actors. This follows the base pattern for activities.

---

[Back to build menu](#build)

In [62]:
def objProd(obj, baseURI,irn, earliestdate,latestdate,datecreated):
           
    artwork = {}
    
    #produced_by property
    artwork["produced_by"] = []
    artwork["produced_by"].append({
                 "id": baseURI + "object/" + irn + "/production",
                "type": "Production",
                "_label": "Production of the Object"})
    
    #carried_out_by property
    carried_out_by = []
    afterfollower = False
    for table in obj["table"]:
        tableName = table["@name"]
        if tableName == "Creator1": 
            if "atom" in table["tuple"]:
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "CreCreatorAfterFollower":
                        afterfollower = True
                    if atom["@name"] == "SummaryData":
                        summarydata = atom["#text"]
                    
            try:   
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "irn":
                        if atom["#text"] not in [2741,10661] or afterfollower == False:
                            carried_out_by.append(
                                {
                                "id":  baseURI + "actor/" + irn,
                                "type": "Actor",
                                "_label": summarydata
                                }
                            )        
            except:
                pass
        
            if tableName == "Creator2":
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "CreCreationCultureOrPeople":
                        carried_out_by.append(
                            {
                            "id":  baseURI + "thesauri/culture/" + atom["#text"],
                            "type": "Actor",
                            "_label": atom["#text"]
                        })
                        
            if len(carried_out_by) > 0:
                artwork["produced_by"].append(
                    {
                    "carried_out_by": carried_out_by
                    })
                
    # timespan property
    timespan = False
    
    
    for date in (earliestdate,latestdate,datecreated):
        if date != "":
            timespan = True
            
    if timespan == True:
        #label
        label = "date unknown"
        if datecreated != "":
            label = datecreated
        elif (earliestdate != "") or (latestdate != ""):
            label = earliestdate + " - " + latestdate
        
        timespanObj = {
               "id": baseURI + "object/" + irn + "/production/timespan",
                "type": "TimeSpan",
                "_label": label,
            }
        
        if earliestdate != "":
            timespanObj["begin_of_the_begin"] = earliestdate
    
        if latestdate != "":
            timespanObj["end_of_the_end"] = latestdate
        
        artwork["produced_by"].append({"timespan" : timespanObj}) 
    
    
    return artwork

In [63]:
  
production = objProd(obj, baseURI,irn, earliestdate,latestdate,datecreated)
frag = {}  
frag.update( min)
frag.update(production)


write2file("production",frag)

print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "produced_by": [
    {
      "id": "https://data.discovernewfields.org/object/93739/production",
      "type": "Production",
      "_label": "Production of the Object"
    },
    {
      "timespan": {
        "id": "https://data.discovernewfields.org/object/93739/production/timespan",
        "type": "TimeSpan",
        "_label": "fabricated 2012",
        "begin_of_the_begin": "2012",
        "end_of_the_end": "2012"
      }
    }
  ]
}


## 4.9 Current Owner and Acquisition <a id="owner"/>

[Linked Art Data Model Documentation](https://linked.art/model/provenance/acquisition/#object-acquisition)

Acquisitions are used to describe the transfer of ownership of an object from one owner to the next. The first owner is typically the artist, who would then transfer it to the second owner, to the third owner and so on. The ownership chain can be expressed by repeating this same pattern with the buyer from one acquisition being the seller in the subsequent one. If the previous owner (e.g. the seller if there is a value exchange) or the subsequent owner (e.g. the buyer) is not known for a particular acquisition, then the reference can be left out from the description.

The acquistion is not necessarily a purchase, it could be a gift, an inheritance or any other method of gaining the right of ownership of an object.

The model encodes this information with an Acquisition part of the overall Provenance Event. The acquisition is the transfer of the right of ownership of an object (referenced in transferred_title_of) from the seller (in transferred_title_from) to the buyer (in transferred_title_to).

Each object has its own Acquisition as part of the provenance event, so if a collector buys three paintings from a dealer, then there would be a single Provenance Event with three Acquisitions, all of which transfer the title of a single painting from the dealer to the collector.

---

[Back to build menu](#build)

In [30]:
def objOwner(obj,baseURI,irn,TitAccessionDate,TitObjectStatus):
    currentowner = False
    artwork = {}
    
    checkObjStatus = ('Accessioned','Partial Accession')
    for status in checkObjStatus:
        if status == TitObjectStatus:
            currentowner = True
    if 'IMA-Owned' in TitObjectStatus:
            currentowner = True
    
    if currentowner == True:
        #current_owner
        artwork["current_owner"] = [
            {
            "id": "http://vocab.getty.edu/ulan/500300517",
            "type": "Group",
            "_label": "Indianapolis Museum of Art at Newfields",
            "classified_as": [
                    {
                        "id": "http://vocab.getty.edu/aat/300312281",
                        "type": "Type",
                        "_label": "museums (institutions)"
                    }
                    ]
            }]
                
        
        if TitAccessionDate != "":
    
            if len(TitAccessionDate) == 4:
                begin = TitAccessionDate + "-01-01T00:00:00.000Z"
                end = TitAccessionDate + "-12-31T00:00:00.000Z"
        
            elif len(TitAccessionDate) == 8:
                begin = TitAccessionDate + "01T00:00:00.000Z"
                end = TitAccessionDate 
                if '-02-' in TitAccessionDate:
                    end = end + "28"
                if ('-01-','-03-','-05-','-07-','-08-','-09-','-10-','-12-') in TitAccessionDate:
                    end = end + "31"
                if ('-04-','-06-','-09-','-11-'):
                    end = end + "30"      
                end = end + "T00:00:00"
         
            elif len(TitAccessionDate) == 10:
                begin = TitAccessionDate + "T00:00:00.000Z"
                end = TitAccessionDate + "T00:00:00.000Z"
        
            else:
                begin = end = ""
        
            timespanjson = [{
                    "id":  baseURI + "object/" + irn + "/IMA-acquisition/timespan", 
                    "type": "TimeSpan", 
                    "_label": TitAccessionDate,
                    }]
            if begin != "":
                timespanjson[0]["begin_of_the_begin"] = begin
            if end != "":
                timespanjson[0]["end_of_the_end"] = end
            
            acquired_title_through = [{
                                "id": baseURI + "object/" + irn + "/IMA-acquisition",
                                "type": "Acquisition",
                                "_label": "IMA at Newfields Acquisition of the Object",
                                "classified_as": [{
                                    "id": "http://vocab.getty.edu/aat/300157782",
                                    "type": "Type",
                                    "_label": "acquisition (collections management)"
                                                }],
                                "took_place_at": [{
                                    "id": "http://vocab.getty.edu/tgn/7012924", 
                                    "type": "Place", 
                                    "_label": "Indianapolis, Indiana"
                                                }],
                                "timespan": timespanjson
                                }]

        
            artwork["current_owner"][0].update({"acquired_title_through" : acquired_title_through})
        
    
    return artwork

In [31]:
acquisition = objOwner(obj,baseURI,irn,TitAccessionDate,TitObjectStatus)

frag = {}
frag.update(core) 
frag.update(acquisition)


write2file("acquisition",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "current_owner": [
    {
      "id": "http://vocab.getty.edu/ulan/500300517",
      "type": "Group",
      "_label": "Indianapolis Museum of Art at Newfields",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312281",
          "type": "Type",
          "_label": "museums (institutions)"
        }
      ],
      "acquired_title_through": [
        {
          "id": "https://data.discovernewfields.org/object/82696/IMA-acquisition",
          "type": "Acquisition",
          "_label": "IMA at Newfields Acquisition of the Object",
          "classified_as": [
            {
              "id": "http://vocab.getty.edu/aat/300157782",
              "type": "Type",
              "_label": "acquisition (collections management)"
            }
          ],
          "took_place_at": 

## 4.10 Custody <a id="curate"/>

[Linked Art Data Model documentation](https://linked.art/model/provenance/custody/#institutional-ownership-departmental-custody)

Objects are owned by legal entities, such as museum organizations or individual people. However there may be more information about which department is responsible within a museum for the curation of the object. This is the division between acquisitions (the legal ownership of the object) and custody (the responsibility for looking after the object). If the department is known, then it should be either part of the Provenance Event in which the object is acquired, or a separate provenance event if the object was not accessioned by a department and later came under their care, or was transferred between departments. In these latter cases, the ownership does not change, only the custody of the object.

The department becomes the `current_keeper` of the object, whereas the institution is the `current_owner`.

---

[Back to build menu](#build)

In [32]:
def objCustody(TitObjectStatus):
    
    currentowner = False
    artwork = {}
    
    checkObjStatus = ('Accessioned','Partial Accession')
    for status in checkObjStatus:
        if status == TitObjectStatus:
            currentowner = True
    if 'IMA-Owned' in TitObjectStatus:
            currentowner = True
            
    if currentowner == False:  
        artwork["current_keeper"] =  {
                "id": "http://vocab.getty.edu/ulan/500300517",
                "type": "Group",
                "_label": "Indianapolis Museum of Art at Newfields",
                "classified_as": [
                    {
                        "id": "http://vocab.getty.edu/aat/300312281",
                        "type": "Type",
                        "_label": "museums (institutions)"
                    }]}
    
    return artwork

In [33]:
custody = objCustody(TitObjectStatus) 

frag = {}
frag.update(core) 
frag.update(custody)


write2file("custody",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes"
}


## 4.11 Membership of Collections and Sets <a id="member"/>

[Linked Art Data Model Documentation](https://linked.art/model/collection/)

There are many use cases for grouping resources together, often of the same class but sometimes of varying types. These use cases are exemplified in the sections below, and range from the set of objects in an auction lot, to dealer inventories and museum collections, exhibitions, a set of related concepts, or the set of people that share a common feature such as gender or nationality.

In order to cover all of the use cases with a consistent pattern, we introduce a new `Set` class from outside of CIDOC-CRM. This avoids issues with sets of resources with different types, and the semantics of the identity of objects and collections. If an equivalent class is added into the core CIDOC-CRM ontology in the future, a new major version of the specification will change to using it.

### Sets 

Core Features

Sets are conceptual groupings, rather than physical ones. The set of objects in a virtual exhibition or simply the set of a person's favorite objects never change their physical state by being part of the Set or not. They are, thus, created by a `Creation`, not by a `Production`.

Like any core resource, Set must have an id and type, are likely to have additional classifications, and can have `Identifiers` and `Names`. They can have statements made about them, and have member resources. These member resources are included via the `member` property rather than `part`, or via `member_of` from the included resource to the `Set`.

---

[Back to build menu](#build)

In [34]:
def objMember(obj,baseURI,PhyCollectionArea):
    artwork={}
    if PhyCollectionArea != "":
        artwork["member_of"] = [{
                 "id": baseURI + "collection/" + PhyCollectionArea,
                    "type": "Set",
                    "_label": "Collection of IMA at Newfields' " + PhyCollectionArea.split("-",1)[1]  + " Department",
                    "classified_as": [
                        {
                            "id": "http://vocab.getty.edu/aat/300025976",
                            "type": "Type",
                            "_label": "collections (object groupings)"
                        }]    
            }]
    return artwork

In [35]:
member = objMember(obj,baseURI,PhyCollectionArea)
    
frag = {}
frag.update(core) 
frag.update(member)

write2file("member",frag)
print(json.dumps(frag,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "member_of": [
    {
      "id": "https://data.discovernewfields.org/collection/907-Contemporary",
      "type": "Set",
      "_label": "Collection of IMA at Newfields' Contemporary Department",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300025976",
          "type": "Type",
          "_label": "collections (object groupings)"
        }
      ]
    }
  ]
}


## 5. Compile Linked Art JSON-LD Representation

Bring the separate descriptions together in a single JSON-LD document

In [36]:
jdoc = {}
jdoc_sections = ["core", "clasf", "homepage", "location", "production", "acquisition", 
                 "dimensions", "custody", "member", "ling","identifiers"]

for section in jdoc_sections:
    if section in vars():
        jdoc.update(vars()[section])
 
jdoc["identified_by"][0].update(names["identified_by"][0])

## 6. Validate the JSON-LD

In [37]:
# https://pynative.com/python-json-validation/

def validateJSON(jsonData):
    try:
        json.loads(jsonData)
    except ValueError as err:
        return False
    return True

isValid = validateJSON(json.dumps(jdoc))

if isValid == True:
    display(HTML("<div class='alert alert-block alert-success'>The JSON-LD file is valid JSON</div"))  
else:
    display(HTML("<div class='alert alert-block alert-danger'>The json-ld file is invalid JSON</div>"))

## 7. View the final Linked Art JSON-LD

The final Linked Art JSON-LD representation of the object is now available:

In [38]:
print(json.dumps(jdoc,indent=2))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/object/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300133025",
      "type": "Type",
      "_label": "works of art"
    },
    {
      "id": "http://vocab.getty.edu/aat/300047090",
      "type": "Type",
      "_label": "sculpture (visual works)"
    }
  ],
  "subject_of": [
    {
      "id": "http://collection.imamuseum.org/artwork/76663",
      "type": "LinguisticObject",
      "_label": "Homepage for the Object",
      "classified_as": [
        {
          "id": "http://vocab/getty.edu/aat/300264578",
          "type": "Type",
          "_label": "Web pages (documents)"
        },
        {
          "id": "http://vocab.getty.edu/aat/300266277",
          "type": "Type",
          "_label": "home pages"
        }
      ],
      "format": "text/html"
    }
  ],
  "current_location": [
    {

In [39]:
f = open("./data/ima/output/json/object_linkedart.json", "w")
f.write(json.dumps(jdoc, indent=2))
f.close() 

## 8. View the Linked Art JSON-LD file

The final JSON-LD file for the selected object is available at:
- [Selected Object as Linked Art](./data/ima/output/json/object_linkedart.json)

View the JSON-LD in the [JSON-LD Playground](https://json-ld.org/playground)

## 9. Transform All Artworks to Linked Art JSON-LD

Define function `getObjDesc` to call the relevant transformation functions

In [75]:
def getObjDesc(obj, functionCall):
    
    desc = {}
    earliestdate = ""
    latestdate = ""
    datecreated = ""
    title = ""
    TitAccessionDate = ""
    provenance = ""
   
    
    for prop in obj["atom"]:      
        propName = prop["@name"]        
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]    
            if propName == "TitAccessionNo":
                titAccessionNo = prop["#text"]
            if propName == "TitMainTitle":
                title = prop["#text"]
            if propName == "TitObjectType":
                titleObjectType = prop["#text"]
            if propName == "SumCreditLine":
                SumCreditLine = prop["#text"]
            if propName == 'CreProvenance':
                provenance = prop["#text"]
            # dates
            if propName == "CreDateCreated":
                datecreated = prop["#text"]
            if propName == "CreEarliestDate":
                earliestdate = prop["#text"]
            if propName == "CreLatestDate":
                latestdate = prop["#text"]
            if propName == "TitAccessionDate":
                TitAccessionDate = prop["#text"]
            if propName == 'TitObjectStatus':
                TitObjectStatus = prop["#text"]
            if propName == "PhyCollectionArea":
                PhyCollectionArea = prop["#text"]
                
    if functionCall == "member":
        desc = objMember(obj,baseURI,PhyCollectionArea)
    if functionCall == "custody":
        desc = objCustody(TitObjectStatus)
    if functionCall == "owner":
        desc = objOwner(obj,baseURI,irn,TitAccessionDate,TitObjectStatus)
    if functionCall == "production":
        desc = objProd(obj, baseURI,irn, earliestdate,latestdate,datecreated)
    if functionCall == "core":
        desc = objCore(obj,baseURI,irn,title) 
    if functionCall == "id":
        desc = objId(obj, baseURI, irn, titAccessionNo)
    if functionCall == "names":
        desc = objNames(obj, baseURI, irn, title)
    if functionCall == "class":
        if "titleObjectType" in vars():
            desc = objClass(obj, baseURI, irn, titleObjectType)
    if functionCall == "home":
            desc = objHome(obj,baseURI,irn)
    if functionCall == "location":
            desc = objLocation(obj,baseURI)
    if functionCall == "ling":
            desc = objLing(obj,baseURI,irn, SumCreditLine, provenance)
        
    return desc



In [76]:
all_linkedart = {}

for obj in allObjects:
    for prop in obj["atom"]:      
        propName = prop["@name"]        
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]   
    core = getObjDesc(obj, "core")
    all_linkedart[irn] = core 
    
    #identifiers
    id = getObjDesc(obj, "id")
    all_linkedart[irn].update(id)
    
    #names
    desc = getObjDesc(obj, "names")
    all_linkedart[irn].update(desc) 
    
    #classification
    desc = getObjDesc(obj, "class")
    if len(desc["classified_as"]) > 0:
        all_linkedart[irn].update(desc)
      
    # homepage
    desc = getObjDesc(obj, "home")
    all_linkedart[irn].update(desc) 
    
    # location
    desc = getObjDesc(obj, "location")
    all_linkedart[irn].update(desc) 
    
    # linguistic objects
    desc = getObjDesc(obj, "ling")
    all_linkedart[irn].update(desc) 
    
    # production
    desc = getObjDesc(obj, "production") 
    all_linkedart[irn].update(desc) 
    
    # ownership
    desc = getObjDesc(obj, "owner")
    all_linkedart[irn].update(desc)  
    
    #custody
    desc = getObjDesc(obj, "custody")
    if "current_keeper" in desc:
        all_linkedart[irn].update(desc)  
    
    # member 
    desc = getObjDesc(obj, "member")
    all_linkedart[irn].update(desc) 
       

Convert Python dictionary to individual Linked Art JSON-LD files for each artwork.

In [77]:
for irn in all_linkedart:
    text_file = open("./data/ima/output/json/all/" + irn + ".json", "wt")
    n = text_file.write(json.dumps(all_linkedart[irn]))
    text_file.close()

f = open("./data/ima/output/json/all/allobjects_linkedart.json", "w")
f.write(json.dumps(all_linkedart, indent=2))
f.close() 

View directory listing of Linked Art JSON-LD files

In [78]:
import os
from IPython.core.display import display, HTML


def fn():       # 1.Get file names from directory
    file_list=os.listdir(r"./data/ima/output/json/all/")
   
    for file in file_list:
        display(HTML("<a href='./data/ima/output/json/all/" + file +"'>" + file + "</a>"))
    
fn()

## 10. Next Steps

Visit the [Linked Art community website](https://linked.art) for further information on the data model and community activities.